In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets transformers accelerate soundfile librosa evaluate jiwer tensorboard gradio


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_17_0", "hu", split="train+validation"
)
common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_11_0", "hu", split="test"
)

print(common_voice)

In [ ]:
common_voice = common_voice.select_columns(["audio", "sentence"])

In [ ]:
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE

TO_LANGUAGE_CODE

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="hungarian", task="transcribe"
)

In [ ]:
common_voice["train"].features

{'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [ ]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [ ]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [ ]:
common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2
)

In [ ]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/48490 [00:00<?, ? examples/s]

In [ ]:
common_voice["train"]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="hungarian", task="transcribe", use_cache=True
)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-cv17-hu",  # name on the HF Hub
    per_device_train_batch_size=64,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=2.5e-5,
    lr_scheduler_type="linear",
    #constant_with_warmup",
    warmup_steps=500,
    max_steps=6000,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=250,
    eval_steps=250,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [ ]:
trainer.train()
#resume_from_checkpoint=True)

In [ ]:
kwargs = {
    "dataset": "Common Voice 17.0",  # a 'pretty' name for the training datase
    "language": "hu",
    "model_name": "Whisper Small Hu CV17",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-base,
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)

**Transcribe part:**

In [ ]:
!pip install faster-whisper

In [ ]:
!ct2-transformers-converter --model sarpba/whisper-tiny-cv17-hu --output_dir whisper-tiny-cv17-hu-foat32 --quantization float32 --low_cpu_mem_usage --copy_files vocab.json --force

In [ ]:
import json

def json_to_txt(json_file, txt_file):
    try:
        with open(json_file, 'r', encoding='utf-8') as jf:
            data = json.load(jf)

        with open(txt_file, 'w', encoding='utf-8') as tf:
            for item in data:
                tf.write(f"{item}\n")

        print(f"A JSON fájl sikeresen konvertálva lett TXT fájlba: {txt_file}")
    except Exception as e:
        print(f"Hiba történt: {e}")

# Használat példa a már feltöltött fájllal
json_file = '/content/whisper-tiny-cv17-hu-int8/vocabulary.json'  # A fájl elérési útja
txt_file = '/content/whisper-tiny-cv17-hu-int8/vocabulary.txt'
json_to_txt(json_file, txt_file)

# Ellenőrizzük, hogy a fájl sikeresen létrejött-e
!ls -l /content/


In [ ]:
!cd /content/whisper-tiny-cv17-hu-int8/
!find ./ -maxdepth 1 -type f -not -path '*/\.*'  -exec md5sum {} \; | tr -d ' ' | jq -R 'split("./") | {(.[1]): (.[0])}' | jq -s 'add' > hash.json

In [ ]:
import json

# A vocab.json fájl beolvasása
with open('/content/whisper-tiny-cv17-hu-foat32/vocab.json', 'r', encoding='utf-8') as f:
    vocab_dict = json.load(f)

# Az új lista létrehozása a dictionary kulcsainak felhasználásával
vocab_list = list(vocab_dict.keys())[:-1]

# A vocabulary.json fájlba írása
with open('/content/vocabulary.json', 'w', encoding='utf-8') as f:
    json.dump(vocab_list, f, ensure_ascii=False, indent=2)